# Состояние (state)

Состояние — это поведенческий паттерн проектирования, который позволяет объектам менять поведение в зависимости от своего состояния. Извне создаётся впечатление, что изменился класс объекта.


## Пример

Вы пишите аналог Jira (систему отслеживания ошибок). В вашей системе, любая созданная заявка должна пройти 3 стадии: новая, реализовывается, завершено. В каждом из этих статусов, активно отдельное поле для комментариев. Требуется реализовать описанный функционал.


In [9]:
from __future__ import annotations
import abc
from typing import Type


class Context:
    """
    Контекст позволяет хранить объект через который выполняются операции, давая возможность заменить его.
    """

    def __init__(self, state_type: Type[AbsState]) -> None:
        self.transition_to(state_type)

    def transition_to(self, state_type: Type[AbsState]) -> None:
        """
        Метод позволяет менять текущее состояние.
        """

        print(f">>> Контекст: Изменен на {state_type.__name__}")
        self._state = state_type(self)

    """
    Взаимодействия с заявкой происходит через контекст.
    """

    def change_to_new(self) -> None:
        self._state.change_to_new()

    def change_to_in_process(self) -> None:
        self._state.change_to_in_process()

    def change_to_final(self) -> None:
        self._state.change_to_final()


class AbsState(abc.ABC):
    def __init__(self, context: Context) -> None:
        # Храним контекст, для перехода между состояниями
        self._context = context

    @abc.abstractmethod
    def change_to_new(self) -> None:
        ...

    @abc.abstractmethod
    def change_to_in_process(self) -> None:
        ...

    @abc.abstractmethod
    def change_to_final(self) -> None:
        ...


class FinalRequestState(AbsState):
    def change_to_new(self) -> None:
        print("Заявка уже в работе, запрещено переводить в предыдущее состояние")

    def change_to_in_process(self) -> None:
        print("Заявка уже в работе, запрещено переводить в предыдущее состояние")

    def change_to_final(self) -> None:
        print("Заявка уже в завершена")


class InProcessRequestState(AbsState):
    def change_to_new(self) -> None:
        print("Заявка уже в работе, запрещено переводить в предыдущее состояние")

    def change_to_in_process(self) -> None:
        print("Заявка уже в работе")

    def change_to_final(self) -> None:
        # Если бы мы не хранили контекст, то не смогли бы поменять его состояние
        self._context.transition_to(FinalRequestState)
        print('Заявка переведена в статус "Завершено"')


class NewRequestState(AbsState):
    def change_to_new(self) -> None:
        print("Заявка уже открыта")

    def change_to_in_process(self) -> None:
        self._context.transition_to(InProcessRequestState)
        print('Заявка переведена в статус "Реализовывается"')

    def change_to_final(self) -> None:
        print('Запрещено закрывать заявку миную статус "Реализовывается"')

In [10]:
context = Context(NewRequestState)
context.change_to_new()
context.change_to_in_process()
context.change_to_final()
context.change_to_final()

>>> Контекст: Изменен на NewRequestState
Заявка уже открыта
>>> Контекст: Изменен на InProcessRequestState
Заявка переведена в статус "Реализовывается"
>>> Контекст: Изменен на FinalRequestState
Заявка переведена в статус "Завершено"
Заявка уже в завершена
